In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-geometric
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=676b5f25b7fc486b10168987a68301209e237f81c8f3b061f42005d22645fd7a
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.

In [ ]:
from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader
import numpy as np
import torch
from tqdm import tqdm
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/My Drive/Notebooks/Pytorch/First_Pytorch_Project/dataset/data.csv')
df.head()

,Source,S_Port,Destination,D_Port,Packets,Bytes,Packets_sent,Bytes_sent,Packets_received,Bytes_received,Rel Start,Duration,Bits_sent_rate,Bits_received_rate,Label
0,172.30.6.255,57041,192.168.240.1,7680,3,198,3,198,0,0,1596.993566,9.007770,175.848184,0.000000,Benign
1,172.30.6.255,55638,35.186.249.84,80,86,67650,34,3483,52,64167,36.448377,22.163850,1257.182304,23160.958050,Benign
2,172.30.15.2348,51682,192.168.121.37,443,95,82309,34,3046,61,79263,5313.787170,4.092096,5954.894509,154958.241400,Benign
3,10.0.2.15,50719,119.254.87.201,80,11,1582,6,705,5,877,25327.813510,0.896225,6293.062568,7828.391308,Malicious
4,172.30.15.1555,50864,54.169.137.62,80,10,1888,6,998,4,890,3555.231347,1.830215,4362.329016,3890.253331,Benign


In [ ]:
from sklearn.model_selection import train_test_split
feature_cols = ['Packets', 'Bytes','Packets_sent','Bytes_sent','Packets_received','Bytes_received','Bits_sent_rate','Bits_received_rate','Duration']
#df = pd.get_dummies(df, prefix='attack',columns=['Label'])
#y=df['Label']
df['Label']=df['Label'].map({'Benign': 0, 'Malicious': 1})
df.drop(['Rel Start','S_Port','D_Port' ],axis='columns', inplace=True)
X_train, X_test = train_test_split(df, test_size = 0.3, random_state = 1)
#X_train, X_test, y_train, y_test = train_test_split(df,y, test_size = 0.3, random_state = 1)
print(X_train['Source'].isnull().values.any()) #check for NAN values

#y_train = y_train.map({'Benign': 0, 'Malicious': 1})
#y_test = y_test.map({'Benign': 0, 'Malicious': 1})
#print(list(X_train['Source'].values))


from sklearn.preprocessing import normalize

#Normal
X_train2=X_train.copy(deep=True)
X_test2=X_test.copy(deep=True)
X_train.drop(['Source','Destination','Label'], axis='columns',inplace=True)
X_test.drop(['Source','Destination','Label'], axis='columns',inplace=True)
X_train= normalize(X_train, norm = 'l1')
X_test = normalize(X_test, norm = 'l1')

X_train = pd.DataFrame(X_train, columns =feature_cols)
X_test = pd.DataFrame(X_test, columns = feature_cols)

X_train['Source']=list(X_train2['Source'].values)
X_train['Destination']=list(X_train2['Destination'].values)
X_train['Label']=list(X_train2['Label'].values)

X_test['Source']=list(X_test2['Source'].values)
X_test['Destination']=list(X_test2['Destination'].values)
X_test['Label']=list(X_test2['Label'].values)

X_train.to_csv('train.csv')
X_test.to_csv('test.csv')
X_train.head()

False


,Packets,Bytes,Packets_sent,Bytes_sent,Packets_received,Bytes_received,Bits_sent_rate,Bits_received_rate,Duration,Source,Destination,Label
0,0.002347,0.464658,0.001257,0.284880,0.001090,0.179778,0.005118,0.037320,0.023552,172.30.6.255,157.240.16.39,0
1,0.002350,0.477663,0.001040,0.117827,0.001310,0.359836,0.004103,0.008849,0.027024,172.30.1.136,172.217.161.4,0
2,0.001253,0.384381,0.000726,0.089185,0.000528,0.295196,0.000890,0.052864,0.174976,192.168.43.224,151.101.153.2,0
3,0.001260,0.466642,0.000658,0.109550,0.000603,0.357092,0.003363,0.014281,0.046551,172.30.6.255,52.220.13.130,0
4,0.001194,0.407213,0.000661,0.093616,0.000533,0.313596,0.000453,0.042010,0.140724,192.168.43.233,13.107.42.254,0


In [ ]:
from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset, Data
import numpy as np
import torch
from tqdm import tqdm
import pickle
import os

class NetworkData(Dataset):
    def __init__(self,root, test=False, transform=None, pre_transform=None):
        #f = open(path_data,'rb')
        self.conn_feat = ['Packets', 'Bytes','Packets_sent','Bytes_sent','Packets_received','Bytes_received','Bits_sent_rate','Bits_received_rate','Duration']
        self.label_cols_oh = ['attack_Benign','attack_Malicious']
        super(NetworkData, self).__init__(root,transform, pre_transform)
        #self.df = filename


    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)
        """
        return 'train.csv'

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""

        return 'not_implemented.pt'

    def download(self):
        pass

    def process(self, n_rows=200):
        self.df = pd.read_csv(self.raw_paths[0])
        print(len(self.df))
        for index, mol in tqdm(self.df.iterrows(), total=self.df.shape[0]):
            # Get node features
            node_feats = self._get_node_features(n_rows=200)
            # Get edge features
            edge_feats = self._get_edge_features(n_rows=200)
            # Get adjacency info
            edge_index = self._get_edge_index(n_rows=200)
            # Get labels info
            label = self._get_labels(mol["Label"])

            # Create data object
            data = Data(x=node_feats,
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label
                        )


            torch.save(data,
                    os.path.join(self.processed_dir,
                                 f'data_{index}.pt'))


    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir,
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir,
                                 f'data_{idx}.pt'))
        return data


    def make_ip_map(self, data):
        unique_ip = np.unique(np.append(data['Source'].to_numpy(),
                                        data['Destination'].to_numpy()))
        return {ip:idx for idx, ip in enumerate(unique_ip)}

    def encode_ip(self, value):
        temp = []
        value=value.split(".")
        value.pop(-1)
        for i in range(len(value)):
          binary=bin(int(value[i])).replace("0b", "") #convert each part of the IP to binary

          #pad any binary less than 8bits to ensure consistency
          if len(binary)!=8:
            padding_n=8-len(binary)

            for j in range(padding_n):
              binary='0'+str(binary)
          value[i]=binary


        value=''.join(value)
        for i in value:
          temp.append(int(i))

        return temp

    def get_ip_feat(self, ip_map):
        ip_data = []
        for ip, idx in ip_map.items():
            ip_data.append(self.encode_ip(ip))

        return ip_data    # = Data.x

    def make_edges(self, data, ip_map):
        src = []
        dst = []
        count = 0
        for _, row in data.iterrows():
            #source ip to connection
            src.append(ip_map[row['Source']])
            dst.append(count)

           #connection dst is common to both IP src

            #destination ip to connection
            src.append(ip_map[row['Destination']])
            dst.append(count)
            count +=1

        return torch.tensor([src, dst]), torch.tensor([dst, src],dtype=torch.float)

    def get_info_conn(self, data, cols):

        return data[cols].values


    def _get_edge_features(self,n_rows):
      edge=[]
      for i in range(1, (len(self.df)//n_rows)+1):
          start_idx = (i-1)*n_rows #assuming i=1, start = (1-1)*200 = 0; end=1*200=200; that is 0:200
          end_idx = i*n_rows
          sample = self.df[start_idx:end_idx]
          edge_feature = self.get_info_conn(sample, self.conn_feat)
          edge.extend(edge_feature)
      edge=np.asarray(edge)
      return torch.tensor(edge,dtype=torch.float)

    def _get_edge_index(self,n_rows):
      index=[]
      for i in range(1, (len(self.df)//n_rows)+1):
          start_idx = (i-1)*n_rows #assuming i=1, start = (1-1)*200 = 0; end=1*200=200; that is 0:200
          end_idx = i*n_rows
          sample = self.df[start_idx:end_idx]
          ip_map = self.make_ip_map(sample) #such as {192.168.4.2:0,192.168.40.42:1, ....etc}
          ip_to_conn, conn_to_ip = self.make_edges(sample, ip_map) #such as
          index.extend(ip_to_conn)

      return index

    def _get_node_features(self, n_rows):
        """
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []
        for i in range(1, (len(self.df)//n_rows)+1):
          node_feats = []
          start_idx = (i-1)*n_rows #assuming i=1, start = (1-1)*200 = 0; end=1*200=200; that is 0:200
          end_idx = i*n_rows
          sample = self.df[start_idx:end_idx]
          ip_map = self.make_ip_map(sample)
          node_feats = self.get_ip_feat(ip_map)
          all_node_feats.extend(node_feats) #copy

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)




In [ ]:
cic_data = NetworkData(root='/content/drive/MyDrive/Notebooks/Pytorch/First_Pytorch_Project/Project/data')
#x=cic_data.get_ip_feat(cic_data.make_ip_map(X_train[:10]))




Processing...


8929


  2%|▏         | 155/8929 [01:51<1:44:55,  1.39it/s]


KeyboardInterrupt: ignored

In [ ]:
len(x)

288

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        #return the indices of a non-zero element
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [ ]:
model = Net(24, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, edge_index)

    # We perform a new round of negative sampling for every training epoch:
    #neg_edge_index = negative_sampling(
    #    edge_index=edge_index, num_nodes=len(x),
    #    num_neg_samples=edge_label_index.size(1))

   # edge_label_index = torch.cat(
   #     [edge_label_index, neg_edge_index],
    #    dim=-1,
   #)
   # edge_labell = torch.cat([
   #    edge_label,
   #     edge_label.new_zeros(neg_edge_index.size(1))
   # ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [ ]:
print(x)
print(edge_index)
model.encode(x, edge_index)

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 1., 1.,  ..., 0., 1., 0.],
        [0., 1., 1.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.]])
tensor([[ 93, 113, 110,  20,  93,  52,  56,  45,  93,   2, 108, 124,  56, 135,
         110,  51,  93,  94,  93, 118, 110, 138, 108, 145,  56,  37,   0,  23,
           0,  23, 110, 105,   0,  23,  93, 130,  93,  99, 110, 152,  56,  35,
           0,  23, 107, 106, 107,  34,  93, 115,  93, 131,  59, 112,   0,  23,
          56, 137, 108, 117, 107,  98,  93,   8,  77,  27,   0,  23,   0,  23,
          69, 125, 107, 114,  93, 151, 107,  46,   0,  23,  56, 156, 107,  47,
         107,  46,   0,  23,  65, 103, 110, 120, 111, 127,   0,  23,  93, 163,
          93, 157,  70, 125, 111, 161,  93,   8,   0,  23,  71, 136,   0,  23,
          91, 162,  93,   1,   0,  23,  87, 104,  56,  45,   0,  23,  86, 144,
         111,  49, 108, 150, 107, 1

RuntimeError: ignored

In [ ]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 10):
    loss = train()
    #val_auc = test(val_data)
    #test_auc = test(test_data)
    #if val_auc > best_val_auc:
        #best_val = val_auc
        #final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    #print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
#f'Test: {test_auc:.4f}')

#print(f'Final Test: {final_test_auc:.4f}')

TypeError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
import torch_geometric
# Define the Graph Classification Model



# Define the Graph Classification Model
class EdgeClassifier(MessagePassing):
    def __init__(self, in_channels, edge_channels, hidden_channels, num_classes):
        super(EdgeClassifier, self).__init__(aggr='mean')  # Use 'mean' aggregation method
        self.edge_lin = nn.Linear(edge_channels, hidden_channels)
        self.node_lin = nn.Linear(in_channels, hidden_channels)
        self.out_lin = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, edge_attr):
        edge_embedding = self.edge_lin(edge_attr)
        x = self.node_lin(x)
        out = self.propagate(edge_index, x=x, edge_attr=edge_embedding)
        out = self.out_lin(out)
        return out

    def message(self, x_j, edge_attr):
        return x_j + edge_attr.unsqueeze(1)

# Generate some dummy data
edge_index = torch.tensor([[0, 1, 2, 2, 3, 4], [1, 2, 0, 3, 1, 2]], dtype=torch.long)
x = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]], dtype=torch.float)
edge_attr = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [0.2, 0.3, 0.4], [0.5, 0.6, 0.7]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1, 0], dtype=torch.long)

# Set up the dataset
dataset = torch_geometric.data.Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# Define the data loader
loader = DataLoader([dataset], batch_size=1)

# Determine the number of classes
num_classes = torch.unique(y).size(0)

# Instantiate the model
model = EdgeClassifier(in_channels=x.size(1), edge_channels=edge_attr.size(1), hidden_channels=64, num_classes=num_classes)

# Set up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
def train():
    model.train()
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out.view(-1, num_classes), data.y)
        loss.backward()
        optimizer.step()

# Train the model
for epoch in range(50):
    train()


ValueError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn import RGCNConv

# Define the Graph Classification Model using RGCN
class EdgeClassifier(nn.Module):
    def __init__(self, in_channels, num_relations, hidden_channels, num_classes):
        super(EdgeClassifier, self).__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels, num_relations, num_bases=30)
        self.conv2 = RGCNConv(hidden_channels, hidden_channels, num_relations, num_bases=30)
        self.lin = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, edge_type):
        x = self.conv1(x, edge_index, edge_type)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_type)
        x = torch.relu(x)
        x = self.lin(x)
        return x

# Generate some dummy data
edge_index = torch.tensor([[0, 1, 2, 2, 3, 4], [1, 2, 0, 3, 1, 2]], dtype=torch.long)
x = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]], dtype=torch.float)
edge_type = torch.tensor([0, 1, 0, 1, 0], dtype=torch.long)
y = torch.tensor([0, 1, 0, 1, 0], dtype=torch.long)

# Set up the dataset
dataset = torch_geometric.data.Data(x=x, edge_index=edge_index, edge_attr=edge_type, y=y)

# Define the data loader
loader = DataLoader([dataset], batch_size=1)

# Instantiate the model
model = EdgeClassifier(in_channels=x.size(1), num_relations=torch.max(edge_type).item() + 1, hidden_channels=64, num_classes=torch.max(y).item() + 1)

# Set up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
def train():
    model.train()
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Train the model
for epoch in range(50):
    train()


IndexError: ignored